In [2]:
!pip install blankly langchain langchain-community langchain-core langchain-google-genai -q

In [37]:
!curl -sL https://github.com/klutometis/build-with-ai-data/archive/refs/heads/main.tar.gz | tar zxvf - --strip-components=1

build-with-ai-data-main/.git-crypt/
build-with-ai-data-main/.git-crypt/.gitattributes
build-with-ai-data-main/.git-crypt/keys/
build-with-ai-data-main/.git-crypt/keys/default/
build-with-ai-data-main/.git-crypt/keys/default/0/
build-with-ai-data-main/.git-crypt/keys/default/0/6C7816ECABE6500B1A4A1B380EFC2ADA85B9BA9C.gpg
build-with-ai-data-main/.gitattributes
build-with-ai-data-main/.gitmodules
build-with-ai-data-main/.pre-commit-config.yaml
build-with-ai-data-main/BUILD
build-with-ai-data-main/WORKSPACE
build-with-ai-data-main/backtest.json
build-with-ai-data-main/blankly.json
build-with-ai-data-main/data.json
build-with-ai-data-main/generate.py
build-with-ai-data-main/keys.json
build-with-ai-data-main/params.py
build-with-ai-data-main/pre-commit/
build-with-ai-data-main/requirements.in
build-with-ai-data-main/requirements.txt
build-with-ai-data-main/run.sh
build-with-ai-data-main/settings.json


In [4]:
import ast
import json
import logging
import pprint
import random
import sys
import textwrap
from datetime import date, datetime
from enum import Enum

import blankly
import langchain
import numpy as np
import pandas as pd
from google.colab import userdata
from langchain.output_parsers.enum import EnumOutputParser
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.tools import tool
from langchain.tools.render import render_text_description
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

TICKER = "TSLA"

logger = logging.getLogger("notebook")
logger.setLevel(logging.INFO)

In [8]:
def get_gemini_model():
    return ChatGoogleGenerativeAI(
        model="gemini-pro",
        google_api_key=userdata.get("gemini"),
        temperature=0,
    )


def plot(data, ticker):
    import plotly.express as px
    import plotly.graph_objects as go

    data["time"] = pd.to_datetime(data["time"], unit="s")

    coefficients = np.polyfit(data.index, data["Account Value (USD)"], deg=1)
    polynomial = np.poly1d(coefficients)

    fig = go.Figure()
    fig = fig.add_trace(
        go.Scatter(
            x=data["time"],
            y=data["Account Value (USD)"],
            mode="lines",
            name="Account Value (USD)",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=data["time"],
            y=polynomial(data.index),
            mode="lines",
            name="Regression",
        )
    )
    fig.update_layout(
        title=f"Time vs. money ({ticker})",
        xaxis_title="Time",
        yaxis_title="USD",
        yaxis=dict(
            tickprefix="$", ticks="outside"
        ),  # Add dollar sign as prefix
        yaxis_tickformat="s",
        width=800,  # Set the width of the plot
        height=600,  # Set the height of the plot
        legend=dict(
            x=0.2,  # Legend x position (0 (left) to 1 (right) within the plotting area)
            y=0.9,  # Legend y position (0 (bottom) to 1 (top) and above)
            xanchor="center",  # Anchor the x position of the legend
            bgcolor="rgba(255, 255, 255, 0.5)",  # Semi-transparent white background
            bordercolor="Black",
            borderwidth=0,
        ),
    )
    fig.show()

In [ ]:
class Action(Enum):
    BUY = 1
    HOLD = 2
    SELL = 3


def init(symbol, state: blankly.StrategyState):
    pass


def price_event(price, symbol, state: blankly.StrategyState):
    # TODO(danenberg): Replace this with a call to LLM.
    match random.choices(list(Action), weights=[1, 1, 1], k=1)[0]:
        case Action.BUY:
            if state.interface.cash >= price:
                state.interface.market_order(symbol, side="buy", size=1)
        case Action.SELL:
            if state.interface.account[TICKER]["available"]:
                state.interface.market_order(symbol, side="sell", size=1)

    logger.info(f"{state.interface.account[TICKER]=}")


exchange = blankly.Alpaca()
strategy = blankly.Strategy(exchange)
strategy.add_price_event(
    price_event, symbol=TICKER, resolution="1w", init=init
)

results = strategy.backtest(to="1y", initial_values={"USD": 10000})
data = results.get_account_history()

INFO: No portfolio name to load specified, defaulting to the first in the file: (example-portfolio). This is fine if there is only one portfolio in use.
INFO:notebook:state.interface.account[TICKER]={'available': 0, 'hold': 0}



Backtesting...


INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 2.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 1.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:notebook:state.interface.account[TICKER]={'available': 0.0, 'hold': 0}
INFO:noteboo

Progress: [##########] 100% Done...


In [ ]:
plot(data, "TSLA")

In [30]:
class Sentiment(Enum):
    POSITIVE = "Positive"
    MIXED = "Mixed"
    NEGATIVE = "Negative"


def read_the_room(news: list[str]):
    sentiment_parser = EnumOutputParser(enum=Sentiment)

    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(
                textwrap.dedent(
                    """\
                You are a news sentiment analyst. Your job is to provide one
                overall sentiment which captures the sentiment of a group of
                headlines.

                Be strongly biased for positive and negative; don't use mixed
                unless absolutely necessary."""
                )
            ),
            HumanMessagePromptTemplate.from_template(
                textwrap.dedent(
                    """\
            Here are the news headlines; what's the overall sentiment?
            {sentiments}

            Avoid mixed; use positive or negative.

            {news}"""
                )
            ),
        ]
    ).partial(sentiments=sentiment_parser.get_format_instructions())

    chain = (
        {"news": RunnablePassthrough()}
        | prompt
        | get_gemini_model()
        | sentiment_parser
    )

    return chain.invoke("\n".join(f"- {news}" for news in news)).value

In [25]:
read_the_room(["Buy TSLA!", "Maybe buy TSLA."])

'Mixed'

In [51]:
global_state = None


@tool
def buy(ticker: str, percent_cash: float, rationale: str, price: float):
    """Buy the stock with some percentage of available cash and give a rationale.

    :param ticker: the name of the stock
    :param percent_cash: percentage of remaining cash to spend
    :param rationale: the rationale for buying the stock
    :param price: price of the stock
    """
    global_state.interface.market_order(
        ticker,
        side="buy",
        size=blankly.trunc(
            (percent_cash * global_state.interface.cash) / price, 2
        ),
    )


@tool
def sell(ticker: str, percent_cash: float, rationale: str, price: float):
    """Sell the stock with some percentage of available cash and give a rationale.

    :param ticker: the name of the stock
    :param percent_cash: percentage of remaining cash to spend
    :param rationale: the rationale for buying the stock
    :param price: the price of the stock
    """
    if available := global_state.interface.account[TICKER]["available"]:
        global_state.interface.market_order(
            ticker,
            side="sell",
            size=blankly.trunc(available * percent_cash, 2),
        )


@tool
def hold(ticker: str, percent_cash: float, rationale: str, price: float):
    """Sell the stock with some percentage of available cash and give a rationale.

    :param ticker: the name of the stock
    :param percent_cash: percentage of remaining cash to spend
    :param rationale: the rationale for buying the stock
    :param stock: the price of the stock
    """
    pass


def init(symbol, state: blankly.StrategyState):
    with open("data.json") as f:
        state.variables["data"] = json.load(f)


def consult_gemini(price, symbol, state: blankly.StrategyState):
    global global_state

    global_state = state
    date = datetime.fromtimestamp(state.time).date().isoformat()
    data = state.variables["data"][date]

    pprint.pp(date)
    pprint.pp(data)

    sentiment = read_the_room(data["news"])
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(
                textwrap.dedent(
                    """\
    You are a stock analyst. Your job is to determine whether to buy, sell or
    hold a stock based on the market sentiment.

    You have the following tools available:

    {tools}"""
                )
            ),
            HumanMessagePromptTemplate.from_template(
                textwrap.dedent(
                    """\
            Should I buy, hold or sell {ticker}? If the sentiment is positive,
            please buy; negative, please sell; mixed, please hold.

            If the RSI dips below 30, go all in! Also, if the MACD value
            is greater than the MACD signal, go all in!

            - Stock price: {price}
            - RSI: {rsi}
            - MACD value: {macd_value}
            - MACD signal: {macd_signal}
            - Sentiment: {sentiment}"""
                )
            ),
        ]
    ).partial(
        tools=render_text_description([buy, sell]),
        sentiment=sentiment,
        price=price,
        rsi=data["rsi"],
        macd_value=data["value"],
        macd_signal=data["signal"],
    )

    model = get_gemini_model().bind(functions=[buy, sell, hold])

    chain = {"ticker": RunnablePassthrough()} | prompt | model
    logger.info(
        call := chain.invoke("TSLA").additional_kwargs["function_call"]
    )
    globals()[call["name"]].invoke(ast.literal_eval(call["arguments"]))


exchange = blankly.Alpaca()
strategy = blankly.Strategy(exchange)
strategy.add_price_event(
    consult_gemini, symbol="TSLA", resolution="1w", init=init
)

results = strategy.backtest(
    start_date="2024-01-24",
    end_date="2024-04-17",
    initial_values={"USD": 10000},
)
data = results.get_account_history()

INFO: No portfolio name to load specified, defaulting to the first in the file: (example-portfolio). This is fine if there is only one portfolio in use.



Backtesting...
'2024-01-24'
{'news': ['Breaking Down the Q4 Earnings Season Scorecard',
          'Breaking Down the Q4 Earnings Season Scorecard',
          'Massive News for Tesla Stock Investors!',
          'Tesla (TSLA) Reports Q4 Earnings: What Key Metrics Have to Say',
          'Tesla Disappoints in Q4, but IBM, ServiceNow Beat',
          'Why Nvidia, Microsoft and other ‘Magnificent Seven’ stocks are on '
          'top in 2024',
          'Tesla (TSLA) Q4 Earnings and Revenues Lag Estimates',
          'Tech has fueled large-cap stocks this year — but not struggling '
          'small caps',
          'Dow ends lower, S&P 500 squeaks by with fourth straight record '
          'close',
          'Massive News for Ford Stock, Rivian Stock, and Tesla Stock '
          'Investors',
          'Stock Market Extends Record Highs, S&P 500 Hits 4,900 On Tech '
          "Rally, Microsoft's $3 Trillion Mark: What's Driving Markets "
          'Wednesday?',
          'Trading Strategi

INFO:notebook:{'name': 'hold', 'arguments': '{"rationale": "Mixed sentiment.", "percent_cash": 1.0, "price": 207.83, "ticker": "TSLA"}'}


'2024-01-31'
{'news': ["Tesla Stock Is Trading Higher After Hours: What's Going On?",
          'Stock-Split Watch: Is Tesla Next?',
          "Elon Musk's Compensation Is 'Tornado Situation' For Tesla: Analyst "
          'Calls For New Plan To Satisfy CEO, Shareholders',
          "You Won't Believe What Tesla CEO Elon Musk Just Said",
          "'Magnificent 7' Tumble As $300B Vanishes On Bleak AI Earnings: "
          'Could The Fed Make Things Worse?',
          'The Better Buy: Tesla Or Toyota? (SA Quant)',
          'This money manager nailed two big stock turning points. These are '
          'his next ‘rocket ship’ ideas.',
          "A Bull Market Is Here. 3 Top Stocks to Buy Like There's No Tomorrow",
          'Dan Ives Called Tesla\'s Latest Earnings a "Train Wreck": Here\'s '
          'His List of Recommendations to Turn the Company Around',
          'Nasdaq Futures Plummet As Alphabet, AMD Disappointments Sour '
          "Sentiment; Spotlight Shifts To Fed Statement, 

INFO:notebook:{'name': 'hold', 'arguments': '{"price": 187.29, "ticker": "TSLA", "rationale": "The sentiment is mixed, the RSI is below 30 and the MACD value is greater than the MACD signal.", "percent_cash": 1.0}'}


...

'2024-04-10'
{'news': ['AI Leaders Retreat: Generational Buying Opportunity?',
          'Why Ascent Solar Technologies Shares Are Trading Lower By Around '
          "50%? Here Are Other Stocks Moving In Wednesday's Mid-Day Session",
          'Tesla Stock Faces Challenges In 2024, 2025: Analyst Highlights This '
          "Key Future Item, 'We Still Think TSLA Warrants A Place In Clients' "
          "Portfolios'",
          'Raise Cash, New Data Shows Hotter Inflation – Momo Gurus Wrong '
          'Again, Bull Market Intact',
          'Cathie Wood Is Dumping Nvidia and Buying Tesla: Is She Making a '
          'Huge Mistake?',
          'Tesla To $205? Here Are 10 Top Analyst Forecasts For Wednesday',
          'Wall Street Analysts Adjust Targets For Nvidia And Tesla Amid '
          'Market Movements',
          "NVIDIA Tops, Tesla Flops Among Magnificent 7 in Q1: Here's Why",
          'A Once-in-a-Generation Investment Opportunity: 1 Artificial '
          'Intelligence (AI) S

INFO:notebook:{'name': 'hold', 'arguments': '{"rationale": "Mixed sentiment.", "percent_cash": 1.0, "price": 171.76, "ticker": "TSLA"}'}


'2024-04-17'
{'news': ['Tesla (TSLA) Sees a More Significant Dip Than Broader Market: Some '
          'Facts to Know',
          "Canoo Has 'A Multi-Billion Revenue Opportunity,' Says Bullish "
          'Analyst',
          'Powell Shifts But Does Not Admit He Was Wrong, Opportunity In AI '
          'Driven Semiconductor Chip Equipment Maker',
          'Q1 Earnings Review',
          'Huge News for Tesla Stock Investors',
          "Tesla Analyst Says Expect 'Fireworks' Into Shareholder Meeting "
          "After EV Giant's Proxy Filing: 'Clock Has Struck Midnight For Musk'",
          'Markets Try to Shake Off April Blues',
          'Billionaire David Tepper Thinks the "Magnificent Seven" Is Really a '
          'Fabulous Five',
          'CyberArk and Manitowoc have been highlighted as Zacks Bull and Bear '
          'of the Day',
          "Tesla Bull Says 'Don't Know…What's Going To Happen To TSLA Stock "
          "Next Week' Ahead Of Earnings Call: 'Much Depends On…'",
     

INFO:notebook:{'name': 'hold', 'arguments': '{"rationale": "The sentiment is mixed.", "percent_cash": 1.0, "ticker": "TSLA", "price": 157.11}'}


Progress: [##########] 100% Done...


In [52]:
plot(data, TICKER)